- This notebook will import all player's roster and advanced log tables into sql.
- The notebook URL_Player_Adv_Log webscrapes and imports tables into the files as CSV

In [1]:
import pandas as pd

import pyodbc

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import os

In [2]:
pyodbc.drivers()

['SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 18 for SQL Server']

In [3]:
# connection nfl_data database link varaibles
servername = '****'
nfl_Data = 'NFL_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [4]:
#create engine url to connect to sql server
engine_nfl_Data = create_engine(f'mssql+pyodbc://@{servername}/{nfl_Data}{trusted_conneciton}{driver}')
engine_nfl_Data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/NFL_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [5]:
conn_engine_nfl_data = engine_nfl_Data.connect()

In [6]:
# union of all season and team to one table for each table
servername = '****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [7]:
#create engine url to connect to sql server
engine_football_data = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')
engine_football_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [8]:
conn_engine_football_data = engine_football_data.connect()

In [9]:
years = range(2010, 2024)

In [10]:
team_li = [
     '49ers',
     'Bears',
     'Bengals',
     'Broncos',
     'Browns',
     'Buccaneers',
     'Buffalos',
     'Cardinals',
     'Chargers',
     'Chiefs',
     'Colts',
     'Commanders',
     'Cowboys',
     'Dolphins',
     'Eagles',
     'Falcons',
     'Giants',
     'Jaguars',
     'Jets',
     'Lions',
     'Packers',
     'Panthers',
     'Patriots',
     'Raiders',
     'Rams',
     'Ravens',
     'Saints',
     'Seahawks',
     'Steelers',
     'Texans',
     'Titans',
     'Vikings'
]

# SQL Import Roster

In [11]:
#import to nfl data
for year in years:
    for team in team_li:
        file_loc = f'NFL_Data_{year}/{team}/{team}_roster'
            
        df = pd.read_csv(file_loc, index_col=0)

        df.to_sql(f'Roster_{team}_{year}', con=conn_engine_nfl_data, if_exists='replace')

In [12]:
# roster to sql union all teams per season
# union all team from nfl_data and save to football_data_year database
all_player_li = []

for year in years:
    #database
    football_db = f'Football_Data_{year}'

    #db connection str
    football_data_eng = create_engine(f'mssql+pyodbc://@{servername}/{football_db}{trusted_conneciton}{driver}')

    football_data_year_conn = football_data_eng.connect()


    players_li = []
    
    for team in team_li:
        players = f'select * from Roster_{team}_{year}'

        players_li.append(players)
    
    players_union = ' union '.join([p for p in players_li])

    all_player_li.append(players_union)

    
    df = pd.read_sql(players_union, con=conn_engine_nfl_data, index_col='index')

    print(year, team, len(df.columns))

    df.to_sql(f'Players', con=football_data_year_conn, if_exists='replace', index='index')




2010 Vikings 14
2011 Vikings 14
2012 Vikings 14
2013 Vikings 14
2014 Vikings 14
2015 Vikings 14
2016 Vikings 14
2017 Vikings 14
2018 Vikings 14
2019 Vikings 14
2020 Vikings 14
2021 Vikings 14
2022 Vikings 14
2023 Vikings 14


In [13]:
#union all players
players_union_all = ' union '.join([a for a in all_player_li])

df_all_pl = pd.read_sql(players_union_all, con=conn_engine_nfl_data, index_col='index')

df_all_pl.to_sql(f'Players', con=conn_engine_football_data, if_exists='replace', index='index')

135

In [14]:
stop

NameError: name 'stop' is not defined

In [ ]:
for year in range(2010,2024):
        #database
    football_db = f'Football_Data_{year}'

    #db connection str
    football_data_eng = create_engine(f'mssql+pyodbc://@{servername}/{football_db}{trusted_conneciton}{driver}')

    football_data_year_conn = football_data_eng.connect()
    
    for team in team_li:
        file_loc = f'NFL_Data_{year}\{team}\{team}_roster'

        df = pd.read_csv(file_loc, index_col=0)

        try:
            df = df.drop(columns=['Unnamed: 0'])
        except:
            pass

        df.to_csv(file_loc, index=0)
        
        df.to_sql('Players', con=football_data_year_conn, if_exists='replace')

In [ ]:
stop

# Import players advanced stats log into sql 

In [ ]:
# imports player advanced log to each season 

years = range(2018,2024)

players_adv_log_name_li = []

for year in years:
    # go to nfl data year data
    year = str(year)

    file_loc_year = f'NFL_Data_{year}/'
        
    os.chdir(file_loc_year)

    
    for team in team_li:
        # got to the team and tis player log
        file_loc_team = f'{team}/Players Log'

        #change folders and get list of files of players
        os.chdir(file_loc_team)

        player_file_list = os.listdir()

        #i want to save all of the players file name as a list to refer back to when
        # I need to union all of the players
        players_adv_log_name_li.append([year]+player_file_list)
        
        for player in player_file_list:
            #get dataframe
            df = pd.read_csv(player, index_col=0)
            # import to sql nfl_data
            df.to_sql(player+'_adv_stats'+'_'+team+'_'+year, con=conn_engine_nfl_data, if_exists='replace')

        os.chdir('..')
        os.chdir('..')
    
    os.chdir('..')

In [ ]:
# get player names list

players_adv_log_name_li = []

years = range(2018,2024)

for year in years:
    year = str(year)

    file_loc_year = f'NFL_Data_{year}/'
        
    os.chdir(file_loc_year)

    
    for team in team_li:
        file_loc_team = f'{team}/Players Log'

        #change folders and get list of files of players
        os.chdir(file_loc_team)

        player_file_list = os.listdir()

        #i want to save all of the players file name as a list to refer back to when
        # I need to union all of the players
        players_adv_log_name_li.append([year]+player_file_list)

        os.chdir('..')
        os.chdir('..')
    
    os.chdir('..')

In [ ]:
players_adv_log_name_li

In [ ]:
stop

#### I need to query each players position, from player advanced log to player roster which has the position information. 
- I'm gettig the position data from the Players table which is all the teams rosters for that season
- I need to get the position from the Players table and 

In [ ]:
#### working on: 4/3/2024

# in sql go get each seasons Players table 
# and use Columns: Player, Team, Pos
# use Players query data to go into each player adv log file

for year in range(2018,2024):
    for team in team_li:
        for player_li in players_adv_log_name_li:
            for player in player_li: